In [1]:
import psycopg2
from psycopg2.extras import RealDictCursor

In [2]:
class Recommender:

    def __init__(self, host, database, user, password, port):
        """

        [작성 2025-12-01]
        LLMtoDatabase class로 저장된 summary 파일 호출하여 기사를 추천.
        
        input: 사용자가 선택한 기사 id (click_id)
        output: 관련 기사 

        """

        # Postgre db 연결
        self.conn = psycopg2.connect(host=host, database=database, user=user, password=password, port=port)
        self.cur = self.conn.cursor(cursor_factory=RealDictCursor)

    def get_similar_articles(self, click_id, k):
        """

        [작성 2025-12-01]
        click_id: 사용자가 클릭한 기사 id
        k: 추천할 기사의 개수

        click_id의 embedding값을 참고하여 코사인 유사도를 계산.
        (pgvector의 코사인 유사도 계산은 정규화 과정을 자동으로 진행)

        반환: json 형식.
        
        """

        query = """
            WITH base AS(
                SELECT embedding
                FROM spnews_summary
                WHERE id = %s
            )
            SELECT
                s.id,
                s.category,
                s.publish_date,
                s.title,
                s.url,
                1 - (s.embedding <=> b.embedding) AS similarity
            FROM spnews_summary s
            CROSS JOIN base b
            WHERE s.id <> %s
            ORDER BY s.embedding <=> b.embedding
            LIMIT %s;
        """

        self.cur.execute(query, (click_id, click_id, k))
        rows = self.cur.fetchall()

        results = []
        for row in rows:
            results.append(
                {
                    "id": row["id"],
                    "category": row["category"],
                    "publish_date": str(row["publish_date"])[:10],
                    "title": row["title"],
                    "url": row["url"]
                }
            )

        return results

    def close(self):
        self.cur.close()
        self.conn.close()

In [3]:
%%time

rec = Recommender(
    host="localhost", 
    database="nvisiaDb", 
    user="postgres", 
    password="postgres1202", 
    port=5432)

similar = rec.get_similar_articles("spnews_101404", k=10)

for article in similar:
    print(article)

rec.close()

{'id': 'spnews_50397', 'category': '경제/산업', 'publish_date': '2022-03-22', 'title': '함남 연포온실농장건설...성토작업-기초공사 한창', 'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=50397'}
{'id': 'spnews_54220', 'category': '경제/산업', 'publish_date': '2022-07-18', 'title': '함경남도 연포온실농장, 온실타일붙이기·살림집 건설', 'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=54220'}
{'id': 'spnews_55668', 'category': '경제/산업', 'publish_date': '2022-09-05', 'title': '北 "화성지구 살림집·연포온실농장·검덕지구 살림집 공사 한창"', 'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=55668'}
{'id': 'spnews_50248', 'category': '경제/산업', 'publish_date': '2022-03-16', 'title': '평양시 화성지구 살림집건설·함남 연포온실농장건설 한창', 'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=50248'}
{'id': 'spnews_48611', 'category': '정치', 'publish_date': '2022-01-28', 'title': '北 김정은, 함남 함주군 연포지구 대규모 채소온실 건설예정지 시찰', 'url': 'https://www.spnews.co.kr/news/articleView.html?idxno=48611'}
{'id': 'spnews_73131', 'category': '경제/산업', 'publish_date': '2023-11-17